<a href="https://colab.research.google.com/github/shahria73/my_learning/blob/master/Implementing_DeepDream_in_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Implementing DeepDream in Keras

We firstly load the InceptionV3 model which tends to produce some of the best visuals. Feel free to try VGG16, VGG19, Xception and ResNet50.

Code obtained and edited from F. Chollet (Created of Keras)
- 
https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/8.2-deep-dream.ipynb

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
from keras.applications import inception_v3
from keras import backend as K

# This setting disables all training specific operations
K.set_learning_phase(0)

# Load InceptionV3
model = inception_v3.InceptionV3(weights = 'imagenet', include_top = False)


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.




We create a dictionary of coefficients quantifying, how much the layer’s activation contributes to the loss you’ll seek to maximize

In [3]:
layer_contributions = {
    'mixed2': 0.7,
    'mixed3': 2.2,
    'mixed4': 1.2,
    'mixed5': .2,
}

Define our tensor that contains the maximized Loss (the weighted sum of the L2 norm of the activations of the layer defined above)

In [6]:
# Map layer names to layer instances
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# loss defined by adding layer contributions
loss = K.variable(0.)

for layer_name in layer_contributions:
  coeff = layer_contributions[layer_name]
  
  # activation gets the layer output
  activation = layer_dict[layer_name].output
  scaling = K.prod(K.cast(K.shape(activation), 'float32'))

  # we add the l2 norm
  loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

Creating the Gradient-Ascent Process


In [7]:
# this is the image or 'dream' :) that is stored in this tensor 
dream = model.input

# Obtains the gradients wrt to the loss
grads = K.gradients(loss, dream)[0]

# Normalizes the gradient 
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# Creates a Keras function to get the value of the loss & gradients wrt to the input
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    """returns the loss and gradient values"""
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    """Implements gradient access for a specified number of iterations"""
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

Implementing the Deep Dream Algorithm

In [12]:
import numpy as np
import scipy
from keras.preprocessing import image
import imageio

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2], 1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    imageio.imwrite(fname, pil_img)

def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

step = 0.01 #Step size for gradient ascent
num_octave = 3 #number of octaves to be run
octave_scale = 1.4 #this is the scale for each ensuing octive will be 1.4 times large than the previous
iterations = 20 #number of gradient ascent operations we execute 
max_loss = 10.0 #our early stoping metric, if loss is > max_loss we break the gradient ascent loop

base_image_path = '/content/image3.jpg'

# Load our image 
img = preprocess_image(base_image_path)

# Initialize a list of tuples for our different images sizes/scales 
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

# Reverse list of shapes, so that they are in increasing order
successive_shapes = successive_shapes[::-1]

# Resize the Numpy array of the image to our smallest scale
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)

    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='/content/' + str(shape) + '.png')
    
save_img(img, fname='/content/final_dream.png')
print("DeepDreaming Complete")

Processing image shape (326, 489)
...Loss value at 0 : 1.5103047
...Loss value at 1 : 1.9625398
...Loss value at 2 : 2.5260818
...Loss value at 3 : 3.0823298
...Loss value at 4 : 3.594502
...Loss value at 5 : 4.1202536
...Loss value at 6 : 4.6098223
...Loss value at 7 : 5.107523
...Loss value at 8 : 5.5600667
...Loss value at 9 : 5.998066
...Loss value at 10 : 6.4316406
...Loss value at 11 : 6.84849
...Loss value at 12 : 7.2640715
...Loss value at 13 : 7.655438
...Loss value at 14 : 8.039218
...Loss value at 15 : 8.41487
...Loss value at 16 : 8.77997
...Loss value at 17 : 9.128268
...Loss value at 18 : 9.461864
...Loss value at 19 : 9.762173
Processing image shape (457, 685)
...Loss value at 0 : 2.5948558
...Loss value at 1 : 3.716318
...Loss value at 2 : 4.6364093
...Loss value at 3 : 5.384675
...Loss value at 4 : 6.096723
...Loss value at 5 : 6.7085786
...Loss value at 6 : 7.2830124
...Loss value at 7 : 7.8145165
...Loss value at 8 : 8.309329
...Loss value at 9 : 8.769828
...Loss val